In [1]:
%matplotlib inline
import os
import shutil
from glob import glob

import numpy as np
import xarray as xr

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
import dask

In [3]:
from ncar_jobqueue import NCARCluster
from dask.distributed import Client
try:
    cluster
    client
except:
    cluster = NCARCluster(cores=1, memory="10 GB", processes=1, walltime="2:00:00")
    cluster.scale(18)
    client = Client(cluster) # Connect this local process to remote workers'
client

Client Scheduler: tcp://10.12.205.200:45029 Dashboard: https://jupyterhub.ucar.edu/dav/user/mgrover/proxy/39371/status,Cluster Workers: 18 Cores: 18 Memory: 180.00 GB


In [4]:
path = '/glade/campaign/cesm/collections/LowWarming/CESM-CAM5-BGC-LowWarming/atm/proc/tseries/daily'
varname = 'TREFHT'
cases = [
    '1pt5degC',
    '1pt5degC_OS',
    '2pt0degC',
]

lon, lat = 5.85573, 52.0407254

for case in cases:
    
    files = sorted(glob(f'{path}/{varname}/b.e11.BRCP26C5CNBDRD.f09_g16.{case}.*.nc'))
    member_ids = [f.split('.')[-6] for f in files]
    
    ds_list = []
    
    for m in set(member_ids):
        print(case, m)
        mem_files = sorted(f for f in files if (f.split('.')[-6] == m))
        with dask.config.set(**{'array.slicing.split_large_chunks': True}):
            dsm = xr.open_mfdataset(mem_files, chunks={'time': 1000}, decode_times=True,)
        ds_list.append(dsm)

    member_id = xr.DataArray(np.arange(len(ds_list)), dims=('member_id'), name='member_id')
    
    with dask.config.set(**{'array.slicing.split_large_chunks': True}):
        ds = xr.concat(ds_list, dim=member_id)

    dsp = ds.interp(lat=lat, lon=lon).reset_coords(['lat', 'lon'], drop=True)
    dsp_mon = dsp.resample(time='M').mean()
    
    for member_id in dsp.member_id.values:
        dsp.TREFHT.sel(member_id=member_id).to_dataframe().to_csv(f'data/cesm-commitment.{case}.TREFHT.{member_id:03d}.csv')        

1pt5degC 009
1pt5degC 003
1pt5degC 002
1pt5degC 006
1pt5degC 012c
1pt5degC 011c
1pt5degC 009c
1pt5degC 010c
1pt5degC 014c
1pt5degC 001
1pt5degC 008
1pt5degC 010
1pt5degC 007
1pt5degC 004
1pt5degC 005
1pt5degC 011
1pt5degC 013c
1pt5degC_OS 003
1pt5degC_OS 002
1pt5degC_OS 012c
1pt5degC_OS 011c
1pt5degC_OS 009c
1pt5degC_OS 010c
1pt5degC_OS 014c
1pt5degC_OS 001
1pt5degC_OS 004
1pt5degC_OS 005
1pt5degC_OS 011
1pt5degC_OS 013c
2pt0degC 009
2pt0degC 003
2pt0degC 002
2pt0degC 006
2pt0degC 012c
2pt0degC 011c
2pt0degC 014c
2pt0degC 001
2pt0degC 008
2pt0degC 010
2pt0degC 007
2pt0degC 004
2pt0degC 005
2pt0degC 011
2pt0degC 013c
